In [ ]:
"""
Demonstration script for the Autonomous Delivery Agent
"""

import os
import sys
sys.path.append(os.path.join(os.path.dirname(__file__), 'src'))

from environment import GridEnvironment
from agent import DeliveryAgent

def main():
    print("=== Autonomous Delivery Agent Demonstration ===\n")

    # Load dynamic map
    env = GridEnvironment(0, 0)
    env.load_from_file('maps/dynamic.map')

    print("Initial Environment:")
    env.visualize()

    # Create agent
    agent = DeliveryAgent(env)

    # Test different algorithms
    algorithms = ['bfs', 'ucs', 'astar', 'hillclimb']
    goal = env.delivery_points[0]

    print(f"\nTesting path from {env.agent_start} to {goal}")
    print("=" * 60)

    for algorithm in algorithms:
        result = agent.plan_path(algorithm, env.agent_start, goal)

        status = "✓ SUCCESS" if result['success'] else "✗ FAILED"
        print(f"{algorithm.upper():<12} | Cost: {result['cost']:>6} | "
              f"Nodes: {result['nodes_expanded']:>6} | Time: {result['planning_time']:>7.4f}s | {status}")

    # Demonstrate dynamic replanning
    print("\n=== Dynamic Replanning Demonstration ===")

    # Add a dynamic obstacle that appears at time step 5
    env.dynamic_obstacles.append((7, 7, 5))

    # Plan initial path with A*
    result = agent.plan_path('astar', env.agent_start, goal)

    if result['success']:
        print("Initial A* path planned successfully")
        print(f"Path cost: {result['cost']}, Length: {len(result['path'])} steps")

        # Simulate execution
        for i, pos in enumerate(result['path']):
            if i == 5:  # At step 5, obstacle appears
                if not env.is_valid_position(*pos, i):
                    print(f"\n🚨 Obstacle detected at step {i}! Position {pos} is blocked.")

                    # Replan from previous position
                    replan_result = agent.replan_for_dynamic_obstacle(
                        result['path'][i-1], goal, 'astar'
                    )

                    if replan_result['success']:
                        print("✅ Replanning successful! New path found.")
                        print(f"New path cost: {replan_result['cost']}")
                        break
                    else:
                        print("❌ Replanning failed!")
                        break

if __name__ == '__main__':
    main()